In [1]:
import pyrealsense2 as rs

In [2]:
deviceDetect = False
ctx = rs.context()
devices = ctx.query_devices()

for dev in devices:
    productName = str(dev.get_info(rs.camera_info.product_id))
    print(productName)
    
    # DS 435 config
    if productName in "0B07":
        EnableColor = True
        EnableInfrared = True
        EnableDepth = True
        deviceDetect = True
        informat = rs.format.y8
        break
    
    # DS 415 config
    elif productName in "0AD3":
        EnableColor = True
        EnableInfrared = True
        EnableDepth = True
        deviceDetect = True
        informat = rs.format.bgr8
        break
        
    # DS 410 config
    elif productName in "0AD2":
        EnableColor = False
        EnableInfrared = True
        EnableDepth = True
        deviceDetect = True
        informat = rs.format.bgr8
        break

if deviceDetect is not True:
    raise Exception("No supported device was found")

0B07


In [3]:
ColorWidth = 1280
ColorHeight = 720
ColorFPS = 30

InfWidth = 1280
InfHeight = 720
InfFPS = 30

In [4]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableColor is True:
    config.enable_stream(rs.stream.color, ColorWidth, ColorHeight, rs.format.bgr8, ColorFPS)

if EnableInfrared is True:
    config.enable_stream(rs.stream.infrared, 1, InfWidth, InfHeight, informat, InfFPS)
    config.enable_stream(rs.stream.infrared, 2, InfWidth, InfHeight, informat, InfFPS)
    
if EnableDepth is True:
    config.enable_stream(rs.stream.depth, InfWidth, InfHeight, rs.format.z16, InfFPS)
    
# Start streaming
cfg = pipeline.start(config)

# Get Video Stream Intrinsics
print("=======================================================")
print("Get Video Stream Intrinsics")
print("=======================================================")
colorStream = cfg.get_stream(rs.stream.color)
colorProfile = colorStream.as_video_stream_profile()
colorIntrin = colorProfile.get_intrinsics()
print("Color Intrinsics:")
print(colorIntrin)
print("")

infraredStream = cfg.get_stream(rs.stream.infrared, 1)
infraredProfile = infraredStream.as_video_stream_profile()
infraredIntrin = infraredProfile.get_intrinsics()
print("Infrared Intrinsics:")
print(infraredIntrin)
print("")

depthStream = cfg.get_stream(rs.stream.depth)
depthProfile = depthStream.as_video_stream_profile()
depthIntrin = depthProfile.get_intrinsics()
print("Depth Intrinsics:")
print(depthIntrin)
print("")

# Get and Apply Depth-to-Stream Extrinsics
print("\n=======================================================")
print("Get and Apply Depth-to-Stream Extrinsics")
print("=======================================================")
print("Color Extrinsics:")
colorExtrin = depthStream.get_extrinsics_to(colorStream)
print(colorExtrin)
print("")
print("Infrared Extrinsics:")
infraredExtrin = depthStream.get_extrinsics_to(infraredStream)
print(infraredExtrin)

# Get Disparity Baseline
print("\n=======================================================")
print("Get Disparity Baseline")
print("=======================================================")
infraredStream2 = cfg.get_stream(rs.stream.infrared, 2)
baseline = infraredStream2.get_extrinsics_to(infraredStream)
print(baseline)

Get Video Stream Intrinsics
Color Intrinsics:
width: 1280, height: 720, ppx: 625.758, ppy: 367.576, fx: 928.179, fy: 926.949, model: Brown Conrady, coeffs: [0, 0, 0, 0, 0]

Infrared Intrinsics:
width: 1280, height: 720, ppx: 639.132, ppy: 362.58, fx: 637.557, fy: 637.557, model: Brown Conrady, coeffs: [0, 0, 0, 0, 0]

Depth Intrinsics:
width: 1280, height: 720, ppx: 639.132, ppy: 362.58, fx: 637.557, fy: 637.557, model: Brown Conrady, coeffs: [0, 0, 0, 0, 0]


Get and Apply Depth-to-Stream Extrinsics
Color Extrinsics:
rotation: [0.999963, 0.0072977, -0.00455206, -0.00728499, 0.99997, 0.00280114, 0.00457237, -0.00276788, 0.999986]
translation: [0.0145757, 0.00041163, 0.000190269]

Infrared Extrinsics:
rotation: [1, 0, 0, 0, 1, 0, 0, 0, 1]
translation: [0, 0, 0]

Get Disparity Baseline
rotation: [1, 0, 0, 0, 1, 0, 0, 0, 1]
translation: [-0.0499424, 0, 0]


In [5]:
pipeline.stop()